In [2]:
import pandas as pd

In [57]:
df1 = pd.read_csv(r'E:\ML Learning\PROJECT_SENTIMENT_ANALYSIS_WHITE_HOUSE\STEP_4_CORRELATION\data_files\roberta_scored_briefings.csv')
df2 = pd.read_csv(r'E:\ML Learning\PROJECT_SENTIMENT_ANALYSIS_WHITE_HOUSE\STEP_4_CORRELATION\data_files\approvalRating2.csv')

In [58]:
#CHANGE DF2 TO DATE TIME
df2['end_date'] = pd.to_datetime(df2['end_date'])

In [72]:
#w will analyze using the year-month as the approval rating is done monthly
df3 = df1.drop(['quarter', 'url'], axis = 1)
#drop duplicates
df3 = df3.drop_duplicates()
df3

,roberta_neg,roberta_neu,roberta_pos,briefing_type,content,date,title
0,0.200637,0.739955,0.059408,Speeches and Remarks,"The White House,1600 Pennsylvania Ave NW,Washi...",2024-06-29,Remarks by President Biden at a Campaign Recep...
1,0.200637,0.739955,0.059408,Speeches and Remarks,"The White House,1600 Pennsylvania Ave NW,Washi...",2024-06-29,Remarks by President Biden at a Campaign Recep...
2,0.200637,0.739955,0.059408,Speeches and Remarks,"The White House,1600 Pennsylvania Ave NW,Washi...",2024-06-18,Remarks as Prepared for Delivery by First Lady...
3,0.200637,0.739955,0.059408,Statements and Releases,"The White House,1600 Pennsylvania Ave NW,Washi...",2024-06-18,Statement from Vice President Kamala Harris on...
4,0.200637,0.739955,0.059408,Speeches and Remarks,"The White House,1600 Pennsylvania Ave NW,Washi...",2024-06-18,Remarks by President Biden at a Campaign Recep...
...,...,...,...,...,...,...,...
10784,0.014081,0.489689,0.496230,Statements and Releases,"The White House,1600 Pennsylvania Ave NW,Washi...",2024-03-27,Statement from President Joe Biden Congratulat...
10785,0.014081,0.489689,0.496230,Speeches and Remarks,"The White House,1600 Pennsylvania Ave NW,Washi...",2024-03-28,Press Call by Vice President Harris On Artific...
10786,0.014081,0.489689,0.496230,Statements and Releases,"The White House,1600 Pennsylvania Ave NW,Washi...",2024-03-28,FACT SHEET: Vice President Harris Announces OM...
10787,0.014081,0.489689,0.496230,Speeches and Remarks,"The White House,1600 Pennsylvania Ave NW,Washi...",2024-03-27,Remarks as Prepared for Delivery by First Lady...


In [61]:
#as we wont use the start date since it starts at the beginning of the month regardless, we will drop it
df2 = df2.drop('start_date', axis =1 )

In [62]:
#as the approval poll is performed monthly, we will analyze the data by month.
#add a year-month column to df2
df2['year_month'] = pd.PeriodIndex(df2['end_date'], freq= 'M')
df2

,approving,disapproving,end_date,unsure,year_month
0,38,58,2024-06-23,4,2024-06
1,39,56,2024-05-23,5,2024-05
2,38,58,2024-04-22,4,2024-04
3,40,55,2024-03-20,5,2024-03
4,38,59,2024-02-20,3,2024-02
5,41,54,2024-01-22,5,2024-01
6,39,59,2023-12-20,2,2023-12
7,37,59,2023-11-20,4,2023-11
8,37,59,2023-10-23,4,2023-10
9,41,58,2023-09-23,1,2023-09


In [67]:
# dropping end_date
df4 = df2.drop('end_date', axis = 1)

In [68]:
df4

,approving,disapproving,unsure,year_month
0,38,58,4,2024-06
1,39,56,5,2024-05
2,38,58,4,2024-04
3,40,55,5,2024-03
4,38,59,3,2024-02
5,41,54,5,2024-01
6,39,59,2,2023-12
7,37,59,4,2023-11
8,37,59,4,2023-10
9,41,58,1,2023-09
